### Import

In [42]:
import numpy as np
import random
import plotly.express as px
import plotly.graph_objects as go
import math
import pandas as pd

### Def module

In [2]:
def load(path):
    with open(path,'r') as f:
        data = f.read().splitlines()
    return np.array(data, np.float64)

def save(data, path):
    with open(path, 'w') as f:
        f.write('\n'.join(map(str, data)))

# the value of number of len will have normal distribution
def random_generator (len, mu, sig):
    nums=[]
    for i in range(len):
        temp = random.normalvariate( mu, sig)
        nums.append(temp)
    return np.array(nums)
# DFT 
def DFT (x, inverse):
    N = len(x)
    n = np.arange(N)
    k = n.reshape((N,1))
    
    if inverse == True:
        e = np.exp( 2j * np.pi *k *n /N)
        y = 1/N * np.dot(e, x)
    else : 
        e = np.exp( -2j * np.pi *k *n /N)
        y = np.dot(e, x)
    return y

def SNR( Amp_signal, Amp_noise):
    rms_amp_signal = (np.sum(Amp_signal**2)/len(Amp_signal))** 0.5
    rms_amp_noise =  (np.sum(Amp_noise**2)/len(Amp_noise))** 0.5
    output = rms_amp_signal/rms_amp_noise
    return output


In [3]:
signal_A = load("./Signal for DSP/Signal A.txt")
signal_B = load("./Signal for DSP/Signal B.txt")
signal_C = load("./Signal for DSP/Signal C.txt")
signal_D = load("./Signal for DSP/Signal D.txt")

In [5]:
print(len(signal_A),len(signal_B), len(signal_C), len(signal_D))

1000 1000 1000 1000


In [54]:
fig = go.Figure()
N = 1000

# len, mu, sig
mu = 0.5
std = 1.5
noise_1 = random_generator(N, mu, std)

x_axis = np.linspace(0 , 1, N)

fig.add_traces(go.Scatter( 
                        x = x_axis, y = noise_1, mode='lines', name= 'Signal_A'))

##### Check value

In [62]:
def check_mean(noise,mu ):
    print('nosie_1 set val mean: ',mu,"\n" 
        'noise_1 actually mean:',np.mean(noise),"\n"
        'accuracy:',1-(abs(np.mean(noise)-mu)) )

In [61]:
check_mean(noise_1, mu=0.5)

nosie_1 set val mean:  0.5 
noise_1 actually mean: 0.4797440420986576 
accuaracy: 0.9797440420986576


##### Check distribution

In [65]:
def check_normal_distribution(noise, mu, std, n):
    noise_df = pd.DataFrame( noise, columns=['val'])
    num_between_interval = noise_df[ (noise_df['val'] >= mu-n* std) & (noise_df['val'] <= mu+n*std) ]
    ratio_between_interval = len(num_between_interval) / len(noise_df)
    if(n==1):
        print('ratio between_interval:',ratio_between_interval,"\n"
              'accuracy:', 1-(0.6827-ratio_between_interval))
    elif(n==2):
        print('ratio between_interval:',ratio_between_interval,"\n"
              'accuracy:', 1-(0.9545-ratio_between_interval))

    elif(n==3):
        print('ratio between_interval:',ratio_between_interval,"\n"
              'accuracy:', 1-(0.9973-ratio_between_interval))

In [66]:
check_normal_distribution(noise_1,mu, std, n=1)

ratio between_interval: 0.663 
accuracy: 0.9803000000000001


### Sampling

In [204]:
# fs = 480, 240, 120, 60 ,30 ,15
def sampling(signal, fs):
    signal_df = pd.DataFrame(signal, columns=['num'])
    sampling_df = pd.DataFrame(np.nan, index=[np.array(signal_df.index)], columns=signal_df.columns)
    for i in range(0,len(signal_df),fs):
        sampling_df.iloc[i] = pd.DataFrame(signal_df.iloc[i])
    return sampling_df

In [236]:
fig = go.Figure()
N = 1000
x_axis = np.linspace(0 , 1, N)

fig.add_traces(go.Scatter( 
                        x = x_axis, y = signal_A, mode='lines', name= 'Signal_A'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 480)['num'], mode='markers', name= 'Sampling_A_fs=480'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 240)['num'], mode='markers', name= 'Sampling_A_fs=240'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 120)['num'], mode='markers', name= 'Sampling_A_fs=120'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 60)['num'], mode='markers', name= 'Sampling_A_fs= 60'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 30)['num'], mode='markers', name= 'Sampling_A_fs= 30'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 15)['num'], mode='markers', name= 'Sampling_A_fs= 15'))

#### Linear complement

In [241]:
import scipy
from scipy.interpolate import interp1d

### signal A

#### 線性補點

In [432]:
x = sampling(signal_A, 480)[sampling(signal_A, 480)['num']>=0].index
mi_df = pd.MultiIndex.from_tuples(x)
x = mi_df.get_level_values(0).tolist()
y = sampling(signal_A, 480)[sampling(signal_A,480)['num']>=0].values
y = y.flatten().tolist()
f = interp1d(x, y, kind='linear')
x_missing_480 = np.arange(0, 960)
y_missing_480 = f(x_missing_480)

In [440]:
x = sampling(signal_A, 240)[sampling(signal_A, 240)['num']>=0].index
mi_df = pd.MultiIndex.from_tuples(x)
x = mi_df.get_level_values(0).tolist()
y = sampling(signal_A, 240)[sampling(signal_A,240)['num']>=0].values
y = y.flatten().tolist()
f = interp1d(x, y, kind='linear')
x_missing_240 = np.arange(0, 960)
y_missing_240 = f(x_missing_240)

In [442]:
x = sampling(signal_A, 120)[sampling(signal_A, 120)['num']>=0].index
mi_df = pd.MultiIndex.from_tuples(x)
x = mi_df.get_level_values(0).tolist()
y = sampling(signal_A, 120)[sampling(signal_A,120)['num']>=0].values
y = y.flatten().tolist()
f = interp1d(x, y, kind='linear')
x_missing_120 = np.arange(0, 960)
y_missing_120 = f(x_missing_120)

In [445]:
x = sampling(signal_A, 60)[sampling(signal_A, 60)['num']>=0].index
mi_df = pd.MultiIndex.from_tuples(x)
x = mi_df.get_level_values(0).tolist()
y = sampling(signal_A, 60)[sampling(signal_A,60)['num']>=0].values
y = y.flatten().tolist()
f = interp1d(x, y, kind='linear')
x_missing_60 = np.arange(0, 960)
y_missing_60 = f(x_missing_60)

In [446]:
x = sampling(signal_A, 30)[sampling(signal_A, 30)['num']>=0].index
mi_df = pd.MultiIndex.from_tuples(x)
x = mi_df.get_level_values(0).tolist()
y = sampling(signal_A, 30)[sampling(signal_A,30)['num']>=0].values
y = y.flatten().tolist()
f = interp1d(x, y, kind='linear')
x_missing_30 = np.arange(0, 990)
y_missing_30 = f(x_missing_30)

In [453]:
x = sampling(signal_A, 15)[sampling(signal_A, 15)['num']>=0].index
mi_df = pd.MultiIndex.from_tuples(x)
x = mi_df.get_level_values(0).tolist()
y = sampling(signal_A, 15)[sampling(signal_A,15)['num']>=0].values
y = y.flatten().tolist()
f = interp1d(x, y, kind='linear')
x_missing_15 = np.arange(0, 990)
y_missing_15 = f(x_missing_15)

#### Visualize

In [456]:
fig = go.Figure()
N = 1000
x_axis = np.linspace(0 , 1, N)

fig.add_traces(go.Scatter( 
                        x = x_axis, y = signal_A, mode='lines', name= 'Signal_A'))

fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 480)['num'], mode='markers', name= 'Sampling_A_fs=480'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = y_missing_480, mode='lines', name= 'LC fs = 480'))

fig.add_traces(go.Scatter( 
                        x = x_axis, y = y_missing_240, mode='lines', name= 'Sampling_A_fs=240'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 240)['num'], mode='markers', name= 'LC fs = 240'))

fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 120)['num'], mode='markers', name= 'Sampling_A_fs=120'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = y_missing_120, mode='lines', name= 'LC fs = 120'))

fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 60)['num'], mode='markers', name= 'Sampling_A_fs= 60'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = y_missing_60, mode='lines', name= 'LC fs= 60'))


fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 30)['num'], mode='markers', name= 'Sampling_A_fs= 30'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = y_missing_30, mode='lines', name= 'LC fs= 30'))

fig.add_traces(go.Scatter( 
                        x = x_axis, y = sampling(signal_A, 15)['num'], mode='markers', name= 'Sampling_A_fs= 15'))
fig.add_traces(go.Scatter( 
                        x = x_axis, y = y_missing_15, mode='lines', name= 'LC fs= 15'))


In [ ]:
x = sampling(signal_A, 480)[sampling(signal_A, 480)['num']>=0].index
mi_df = pd.MultiIndex.from_tuples(x)
x = mi_df.get_level_values(0).tolist()
y = sampling(signal_A, 480)[sampling(signal_A,480)['num']>=0].values
y = y.flatten().tolist()
f = interp1d(x, y, kind='linear')
x_missing = np.arange(0, 960)
y_missing = f(x_missing)